In [20]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import KFold
import utils
from sklearn.feature_extraction.text import TfidfVectorizer

In [60]:
df_train = pd.read_csv("C:/Users/JayZheng/Documents/Data Science/data mining/home-depot/data/train_clean.csv",encoding="ISO-8859-1")
df_test = pd.read_csv("C:/Users/JayZheng/Documents/Data Science/data mining/home-depot/data/test_clean.csv",encoding="ISO-8859-1")
df_desc = pd.read_csv("C:/Users/JayZheng/Documents/Data Science/data mining/home-depot/data/product_descriptions_clean.csv")
df_attr = pd.read_csv("C:/Users/JayZheng/Documents/Data Science/data mining/home-depot/data/attributes_clean.csv",encoding = "ISO-8859-1")

In [61]:
df_train = df_train.drop(df_train.columns[0],axis = 1)
df_test = df_test.drop(df_test.columns[0],axis = 1)
numtrain = df_train.shape[0]

In [62]:
df_train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,simpson strongti 12 gaug angl,angl bracket,3.00
1,3,100001,simpson strongti 12 gaug angl,l bracket,2.50
2,9,100002,behr premium textur deckov 1 gal. sc 141 tugbo...,deck,3.00
3,16,100005,delta vero 1 handl shower faucet trim kit chro...,rain shower head,2.33
4,17,100005,delta vero 1 handl shower faucet trim kit chro...,shower faucet,2.67


In [63]:
df_brand = df_attr[df_attr.name == "mfg brand name"][["product_uid", "value"]].rename(columns={"value": "brand"})

In [64]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
df_all = pd.merge(df_all, df_desc, how='left', on='product_uid')

In [65]:
for var in df_all.keys():
    df_all[var]=df_all[var].fillna("na")

In [67]:
attr_all = df_attr[df_attr.name != 'mfg brand name'][['product_uid', 'value']].rename(columns={'value': 'attr'})
attr_all['attr'] = attr_all['attr'].apply(lambda x: x if isinstance(x, str) else 'None')
attr_all = attr_all.groupby('product_uid')['attr'].apply(lambda x: ' '.join(x))
attr_all = pd.DataFrame(attr_all)
attr_all = attr_all.reset_index()

In [68]:
attr_all_title = df_attr[df_attr.name != 'mfg brand name'][['product_uid', 'name']].rename(columns={'name': 'attr_title'})
attr_all_title['attr_title'] = attr_all_title['attr_title'].apply(lambda x: x if isinstance(x, str) else 'None')
attr_all_title = attr_all_title.groupby('product_uid')['attr_title'].apply(lambda x: ' '.join(x))
attr_all_title = pd.DataFrame(attr_all_title)
attr_all_title = attr_all_title.reset_index()

In [15]:
#attr_all_title

In [69]:
df_all = pd.merge(df_all, attr_all, how='left', on='product_uid')
df_all = pd.merge(df_all, attr_all_title, how='left', on='product_uid')

In [70]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x: len(str(x).split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x: len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x: len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x: len(str(x).split())).astype(np.int64)
df_all['len_of_attr'] = df_all['attr'].map(lambda x: len(str(x).split())).astype(np.int64)
df_all['len_of_attr_title'] = df_all['attr_title'].map(lambda x: len(str(x).split())).astype(np.int64)

In [71]:
df_all.head()

,id,product_title,product_uid,relevance,search_term,brand,product_description,attr,attr_title,len_of_query,len_of_title,len_of_description,len_of_brand,len_of_attr,len_of_attr_title
0,2,simpson strongti 12 gaug angl,100001,3,angl bracket,simpson strong-ti,angl make joint stronger provid consis10t stra...,versatil connector for various 90 deg. connect...,bullet 01 bullet 02 bullet 03 bullet 04 bullet...,2,5,85,2,68,31
1,3,simpson strongti 12 gaug angl,100001,2.5,l bracket,simpson strong-ti,angl make joint stronger provid consis10t stra...,versatil connector for various 90 deg. connect...,bullet 01 bullet 02 bullet 03 bullet 04 bullet...,2,5,85,2,68,31
2,9,behr premium textur deckov 1 gal. sc 141 tugbo...,100002,3,deck,behr premium textur deckov,behr premium textur deckov innov solid color c...,brush roller spray 6.63 in. 7.76 in. 6.63 in. ...,applic method assembl depth in. assembl h8 in....,1,12,113,4,147,70
3,16,delta vero 1 handl shower faucet trim kit chro...,100005,2.33,rain shower head,delta,updat bathroom delta vero singlehandl shower f...,combo tub and shower no includ the trim kit on...,bath faucet type built-in water filter bullet ...,3,11,64,1,105,65
4,17,delta vero 1 handl shower faucet trim kit chro...,100005,2.67,shower faucet,delta,updat bathroom delta vero singlehandl shower f...,combo tub and shower no includ the trim kit on...,bath faucet type built-in water filter bullet ...,2,11,64,1,105,65


In [73]:
df_all['query_in_title'] = df_all.apply(lambda x: utils.counter_appearance(x['search_term'].split(), x['product_title'].split()), axis=1)
df_all['query_in_description'] = df_all.apply(lambda x: utils.counter_appearance(x['search_term'].split(), x['product_description'].split()), axis=1)
df_all['query_in_attr'] = df_all.apply(lambda x: utils.counter_appearance(str(x['search_term']).split(), str(x['attr']).split()), axis=1)
df_all['query_in_attr_title'] = df_all.apply(lambda x: utils.counter_appearance(str(x['search_term']).split(), str(x['attr_title']).split()), axis=1)
#df_all['query_last_word_in_title'] = df_all.apply(lambda x: utils.counter_appearance(x['search_term'].split()[-1], x['product_title'].split()), axis=1)
df_all['query_last_word_in_description'] = df_all.apply(lambda x: utils.counter_appearance(x['search_term'].split()[-1], x['product_description'].split()), axis=1)
df_all['query_last_word_in_attr'] = df_all.apply(lambda x: utils.counter_appearance(str(x['search_term']).split()[-1], str(x['attr']).split()), axis=1)
df_all['query_last_word_in_attr_title'] = df_all.apply(lambda x: utils.counter_appearance(str(x['search_term']).split()[-1], str(x['attr_title']).split()), axis=1)
df_all['word_in_title'] = df_all.apply(lambda x: utils.counter_appear_times(x['search_term'].split(), x['product_title'].split()), axis=1)
df_all['word_in_description'] = df_all.apply(lambda x: utils.counter_appear_times(x['search_term'].split(), x['product_description'].split()), axis=1)
df_all['word_in_attr'] = df_all.apply(lambda x: utils.counter_appear_times(str(x['search_term']).split(), str(x['attr']).split()), axis=1)
df_all['word_in_attr_title'] = df_all.apply(lambda x: utils.counter_appear_times(str(x['search_term']).split(), str(x['attr_title']).split()), axis=1)
df_all['word_in_brand'] = df_all.apply(lambda x: utils.counter_appear_times(x['search_term'].split(), x['brand'].split()), axis=1)

In [76]:
desc_tf, desc_idf, desc_length, desc_ave_length = utils.tfidf(df_all, 'product_description')
df_all['desc_BM25_score'] = df_all.apply(lambda x: utils.BM25_score(x, desc_tf, desc_idf, desc_length, desc_ave_length), axis=1)
attr_tf, attr_idf, attr_length, attr_ave_length = utils.tfidf(df_all, 'attr')
df_all['attr_BM25_score'] = df_all.apply(lambda x: utils.BM25_score(x, attr_tf, attr_idf, attr_length, attr_ave_length), axis=1)
title_tf, title_idf, title_length, title_ave_length = utils.tfidf(df_all, 'product_title')
df_all['title_BM25_score'] = df_all.apply(lambda x: utils.BM25_score(x, title_tf, title_idf, title_length, title_ave_length), axis=1)
attr_title_tf, attr_title_idf, attr_title_length, attr_title_ave_length = utils.tfidf(data, 'attr_title')
df_all['attr_title_BM25_score'] = df_all.apply(lambda x: utils.BM25_score(x, attr_title_tf, attr_title_idf, attr_title_length, attr_title_ave_length), axis=1)
brand_tf, brand_idf, brand_length, brand_ave_length = utils.tfidf(data, 'brand')
df_all['brand_BM25_score'] = df_all.apply(lambda x: utils.BM25_score(x, brand_tf, brand_idf, brand_length, brand_ave_length), axis=1)

processing  1000 th documents(tf)
processing  2000 th documents(tf)
processing  3000 th documents(tf)
processing  4000 th documents(tf)
processing  5000 th documents(tf)
processing  6000 th documents(tf)
processing  7000 th documents(tf)
processing  8000 th documents(tf)
processing  9000 th documents(tf)
processing  10000 th documents(tf)
processing  11000 th documents(tf)
processing  12000 th documents(tf)
processing  13000 th documents(tf)
processing  14000 th documents(tf)
processing  15000 th documents(tf)
processing  16000 th documents(tf)
processing  17000 th documents(tf)
processing  18000 th documents(tf)
processing  19000 th documents(tf)
processing  20000 th documents(tf)
processing  21000 th documents(tf)
processing  22000 th documents(tf)
processing  23000 th documents(tf)
processing  24000 th documents(tf)
processing  25000 th documents(tf)
processing  26000 th documents(tf)
processing  27000 th documents(tf)
processing  28000 th documents(tf)
processing  29000 th document

AttributeError: 'float' object has no attribute 'split'

In [ ]:
df_brand = pd.unique(df_all.brand.ravel())
d = {}
i = 1000
for s in df_brand:
    d[s] = i
    i += 3
df_all['brand_feature'] = df_all['brand'].map(lambda x: d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x: len(x))

In [82]:
attr_tf, attr_idf, attr_length, attr_ave_length = utils.tfidf(df_all, 'attr')

AttributeError: 'float' object has no attribute 'split'

In [90]:
df_attr = pd.read_csv("C:/Users/JayZheng/Documents/Data Science/data mining/home-depot/data/attributes.csv")

In [91]:
df_attr

,product_uid,name,value
0,100001.0,Bullet01,Versatile connector for various 90° connection...
1,100001.0,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001.0,Bullet03,Help ensure joints are consistently straight a...
3,100001.0,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001.0,Bullet05,Made from 12-Gauge steel
5,100001.0,Bullet06,Galvanized for extra corrosion resistance
6,100001.0,Bullet07,Install with 10d common nails or #9 x 1-1/2 in...
7,100001.0,Gauge,12
8,100001.0,Material,Galvanized Steel
9,100001.0,MFG Brand Name,Simpson Strong-Tie


AttributeError: 'list' object has no attribute 'split'

,product_uid,product_description
0,100001,angl make joint stronger provid consis10t stra...
1,100002,behr premium textur deckov innov solid color c...
2,100003,classic architectur meet contemporari design e...
3,100004,grape solar 265 watt. polycrystallin pv solar ...
4,100005,updat bathroom delta vero singlehandl shower f...


,id,product_title,product_uid,relevance,search_term,product_description,brand,product_color
0,2,simpson strongti 12 gaug angl,100001,3,angl bracket,angl make joint stronger provid consis10t stra...,Simpson Strong-Tie,unknown
1,3,simpson strongti 12 gaug angl,100001,2.5,l bracket,angl make joint stronger provid consis10t stra...,Simpson Strong-Tie,unknown
2,9,behr premium textur deckov 1 gal. sc 141 tugbo...,100002,3,deck,behr premium textur deckov innov solid color c...,BEHR Premium Textured DeckOver,Browns / Tans
3,9,behr premium textur deckov 1 gal. sc 141 tugbo...,100002,3,deck,behr premium textur deckov innov solid color c...,BEHR Premium Textured DeckOver,Tugboat
4,16,delta vero 1 handl shower faucet trim kit chro...,100005,2.33,rain shower head,updat bathroom delta vero singlehandl shower f...,Delta,Chrome
5,16,delta vero 1 handl shower faucet trim kit chro...,100005,2.33,rain shower head,updat bathroom delta vero singlehandl shower f...,Delta,Chrome
6,17,delta vero 1 handl shower faucet trim kit chro...,100005,2.67,shower faucet,updat bathroom delta vero singlehandl shower f...,Delta,Chrome
7,17,delta vero 1 handl shower faucet trim kit chro...,100005,2.67,shower faucet,updat bathroom delta vero singlehandl shower f...,Delta,Chrome
8,18,whirlpool 19 cu.ft. rang convect microwav stai...,100006,3,convect otr,achiev delici result effortless whirlpool over...,Whirlpool,Stainless Steel
9,18,whirlpool 19 cu.ft. rang convect microwav stai...,100006,3,convect otr,achiev delici result effortless whirlpool over...,Whirlpool,Stainless
